# Figure of 8 to be followed

In [23]:
# 1. Run on prompt
# daniel@simbox /opt/carla-simulator $ ./CarlaUE4.sh 

# 2. boilerplate code
import carla 
import math 
import random 
import time 
import carla_helpers as helpers

# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

## Set spectator position

In [2]:
import carla_helpers as helpers
# get view
# transform = helpers.get_spectator_transform(world)
# set view
transform = helpers.set_spectator_transform(world, (-56.26, 4.20, 554.04), (-87.06, 54.52, 0))


=== Spectator Transform Set ===
Location: x=-56.26, y=4.20, z=554.04
Rotation: pitch=-87.06, yaw=54.52, roll=0.00


## Get figure of 8 for town 04

In [3]:
# helper functions
def validate_figure8_waypoints(world):
    """Get and validate waypoints for figure-8 track."""
    #road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    # road_sequence = [35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    road_sequence = [42, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47, 48, 49, 50]
    target_lane_id = -3
    waypoint_spacing = 1.0
    
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(waypoint_spacing)
    
    # Filter and sort waypoints
    road_waypoints = {}
    for road_id in road_sequence:
        road_waypoints[road_id] = [wp for wp in waypoints 
                                 if wp.road_id == road_id and wp.lane_id == target_lane_id]
        road_waypoints[road_id].sort(key=lambda x: x.s)
        
        print(f"Road {road_id}: {len(road_waypoints[road_id])} waypoints")
        
        # Validate waypoint spacing
        if len(road_waypoints[road_id]) > 1:
            first = road_waypoints[road_id][0].transform.location
            last = road_waypoints[road_id][-1].transform.location
            distance = ((last.x - first.x)**2 + 
                       (last.y - first.y)**2 + 
                       (last.z - first.z)**2)**0.5
            print(f"Road length: {distance:.1f}m")
    
    return road_waypoints

# Usage:
eight_lane_roads = validate_figure8_waypoints(world)
# eight_lane_roads

def place_markers_on_lane(world, eight_lane_roads, target_lane_id=-3):
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(2.0)
    
    # Filter waypoints for target roads and lane
    target_waypoints = [wp for wp in waypoints 
                       if wp.road_id in eight_lane_roads 
                       and wp.lane_id == target_lane_id]
    
    # Place markers
    markers = []
    marker_color = carla.Color(r=255, g=0, b=0)  # Red markers
    marker_life_time = 20.0  # Markers will persist for 20 seconds
    
    for wp in target_waypoints:
        location = wp.transform.location
        # Draw a sphere at each waypoint
        marker = world.debug.draw_point(
            location,
            size=0.1,  # 10cm sphere
            color=marker_color,
            life_time=marker_life_time
        )
        markers.append(marker)
    
    print(f"Placed {len(markers)} markers on lane {target_lane_id} "
          f"across {len(eight_lane_roads)} roads")
    
    return markers, target_waypoints

# Usage:
markers, target_waypoints = place_markers_on_lane(world, eight_lane_roads)


Road 42: 34 waypoints
Road length: 33.0m
Road 35: 286 waypoints
Road length: 231.0m
Road 36: 82 waypoints
Road length: 81.0m
Road 37: 41 waypoints
Road length: 37.3m
Road 38: 246 waypoints
Road length: 236.5m
Road 39: 134 waypoints
Road length: 133.0m
Road 40: 237 waypoints
Road length: 236.3m
Road 41: 210 waypoints
Road length: 194.3m
Road 43: 33 waypoints
Road length: 32.2m
Road 45: 586 waypoints
Road length: 492.9m
Road 46: 139 waypoints
Road length: 141.1m
Road 47: 114 waypoints
Road length: 113.0m
Road 48: 63 waypoints
Road length: 62.0m
Road 49: 40 waypoints
Road length: 38.8m
Road 50: 178 waypoints
Road length: 155.3m
Placed 1214 markers on lane -3 across 15 roads


## Set spectator at first figure-of-eight waypoint

In [4]:
#type(target_waypoints) # list
#target_waypoints[0] # carla.Waypoint object
#dir(target_waypoints[0])
new_spectator_transfrom = target_waypoints[0].transform
x = new_spectator_transfrom.location.x
y = new_spectator_transfrom.location.y
z = new_spectator_transfrom.location.z
pitch = new_spectator_transfrom.rotation.pitch
yaw = new_spectator_transfrom.rotation.yaw
roll = new_spectator_transfrom.rotation.roll
transform = helpers.set_spectator_transform(world, (x, y, z), (pitch, yaw, roll))



=== Spectator Transform Set ===
Location: x=248.67, y=-367.73, z=0.00
Rotation: pitch=0.00, yaw=-0.30, roll=0.00


## Visualise roads one at a time

In [25]:
import time

def visualize_roads_one_by_one(world, eight_lane_roads, target_lane_id=-3, delay=5):
    """
    Visualize roads one by one by placing markers on each road sequentially.
    
    Parameters:
    - world: CARLA world object.
    - eight_lane_roads: Dictionary containing waypoints for each road.
    - target_lane_id: Lane ID to visualize (default is -3).
    - delay: Time delay (in seconds) between visualizing each road.
    """
    marker_color = carla.Color(r=255, g=0, b=0)  # Red markers
    marker_life_time = delay + 2  # Markers will persist slightly longer than the delay
    
    for road_id, waypoints in eight_lane_roads.items():
        print(f"Visualizing Road {road_id}...")
        
        # Place markers on the current road
        markers = []
        for wp in waypoints:
            location = wp.transform.location
            marker = world.debug.draw_point(
                location,
                size=0.2,  # Slightly larger sphere for better visibility
                color=marker_color,
                life_time=marker_life_time
            )
            markers.append(marker)
        
        print(f"Placed {len(markers)} markers on Road {road_id}.")
        
        # Wait for the specified delay before moving to the next road
        time.sleep(delay)
        
        # Clear markers (optional, as they will disappear after life_time)
        # markers.clear()  # Not strictly necessary due to life_time
        
    print("Finished visualizing all roads.")

visualize_roads_one_by_one(world, eight_lane_roads, delay=5)    

Visualizing Road 42...
Placed 34 markers on Road 42.
Visualizing Road 35...
Placed 286 markers on Road 35.
Visualizing Road 36...
Placed 82 markers on Road 36.
Visualizing Road 37...
Placed 41 markers on Road 37.
Visualizing Road 38...
Placed 246 markers on Road 38.
Visualizing Road 39...
Placed 134 markers on Road 39.
Visualizing Road 40...
Placed 237 markers on Road 40.
Visualizing Road 41...
Placed 210 markers on Road 41.
Visualizing Road 43...
Placed 33 markers on Road 43.
Visualizing Road 45...
Placed 586 markers on Road 45.
Visualizing Road 46...
Placed 139 markers on Road 46.
Visualizing Road 47...
Placed 114 markers on Road 47.
Visualizing Road 48...
Placed 63 markers on Road 48.
Visualizing Road 49...
Placed 40 markers on Road 49.
Visualizing Road 50...
Placed 178 markers on Road 50.
Finished visualizing all roads.


## Inspect Road ID 42
To make sure it should be on the route.

In [21]:
import carla

def inspect_road_and_place_spectator(world, eight_lane_roads, road_id=42, waypoint_index=0):
    """
    Inspect a specific road by placing the spectator near or at a waypoint of that road.
    
    Parameters:
    - world: CARLA world object.
    - eight_lane_roads: Dictionary containing waypoints for each road.
    - road_id: The road ID to inspect (default is 42).
    - waypoint_index: Index of the waypoint to place the spectator at (default is 0).
    """
    # Get the spectator object
    spectator = world.get_spectator()
    
    # Check if the road_id exists in the dictionary
    if road_id not in eight_lane_roads:
        print(f"Road ID {road_id} not found in the provided waypoints.")
        return
    
    # Get the waypoints for the specified road
    waypoints = eight_lane_roads[road_id]
    
    # Check if the waypoint_index is valid
    if waypoint_index >= len(waypoints):
        print(f"Waypoint index {waypoint_index} is out of range for Road ID {road_id}.")
        return
    
    # Get the target waypoint
    target_waypoint = waypoints[waypoint_index]
    target_location = target_waypoint.transform.location
    
    # Set the spectator's transform to the waypoint's transform
    spectator_transform = carla.Transform(
        target_location + carla.Location(z=20),  # Move the spectator 20 meters above the waypoint
        carla.Rotation(pitch=-90)  # Look directly down
    )
    spectator.set_transform(spectator_transform)
    
    print(f"Spectator placed at Waypoint {waypoint_index} on Road ID {road_id}.")
    
    # Optional: Visualize the road with markers
    marker_color = carla.Color(r=0, g=255, b=0)  # Green markers for better visibility
    marker_life_time = 30.0  # Markers will persist for 30 seconds
    
    for wp in waypoints:
        world.debug.draw_point(
            wp.transform.location,
            size=0.2,  # 20cm sphere
            color=marker_color,
            life_time=marker_life_time
        )
    
    print(f"Placed {len(waypoints)} markers on Road ID {road_id} for inspection.")

# road_sequence = [35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
# Assuming `world` and `eight_lane_roads` are already defined
#inspect_road_and_place_spectator(world, eight_lane_roads, road_id=42, waypoint_index=0)    
# inspect_road_and_place_spectator(world, eight_lane_roads, road_id=35, waypoint_index=0)    
inspect_road_and_place_spectator(world, eight_lane_roads, road_id=50, waypoint_index=0)  
inspect_road_and_place_spectator(world, eight_lane_roads, road_id=42, waypoint_index=0)  

Spectator placed at Waypoint 0 on Road ID 50.
Placed 178 markers on Road ID 50 for inspection.
Spectator placed at Waypoint 0 on Road ID 42.
Placed 34 markers on Road ID 42 for inspection.


## Helper functions

In [54]:
def get_figure8_waypoints(world):
    # road_sequence = [42, 35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50]
    road_sequence = [35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50]
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(1.0)
    
    all_waypoints = []
    for road_id in road_sequence:
        road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == -3]
        road_waypoints.sort(key=lambda x: x.s)
        all_waypoints.extend(road_waypoints)
        
    return all_waypoints

def set_spectator_camera_following_car(world, vehicle):
    """
    Position the spectator 3 meters above and 5 meters behind the vehicle.
    
    Parameters:
    - world: CARLA world object.
    - vehicle: CARLA vehicle object.
    
    Returns:
    - spectator: CARLA spectator object.
    """
    spectator = world.get_spectator()
    vehicle_transform = vehicle.get_transform()

    # Get the vehicle's forward vector (direction it's facing)
    forward_vector = vehicle_transform.get_forward_vector()

    # Calculate the offset position: 5 meters behind and 3 meters above the vehicle
    offset_location = vehicle_transform.location + carla.Location(
        x=-5 * forward_vector.x,  # 5 meters behind along the forward axis
        y=-5 * forward_vector.y,  # 5 meters behind along the forward axis
        z=3  # 3 meters above
    )

    # Set the spectator transform with a fixed pitch for a better view
    spectator.set_transform(carla.Transform(
        offset_location,
        carla.Rotation(pitch=-15, yaw=vehicle_transform.rotation.yaw, roll=0)  # Slightly tilted down
    ))
    return spectator

def compute_control(vehicle, target_wp, target_speed=10):  # Reduced target speed to 10 km/h
    """
    Compute vehicle control to reach the target waypoint at a specified speed.
    
    Parameters:
    - vehicle: CARLA vehicle object.
    - target_wp: Target waypoint.
    - target_speed: Desired speed in km/h (default is 10 km/h).
    
    Returns:
    - control: CARLA VehicleControl object.
    """
    control = carla.VehicleControl()

    # Get current state
    current_transform = vehicle.get_transform()
    current_velocity = vehicle.get_velocity()
    speed = 3.6 * math.sqrt(current_velocity.x**2 + current_velocity.y**2)  # Convert to km/h

    # Steering control
    forward = current_transform.get_forward_vector()
    target_vector = target_wp.transform.location - current_transform.location
    forward_dot = forward.x * target_vector.x + forward.y * target_vector.y
    right_dot = -forward.y * target_vector.x + forward.x * target_vector.y
    steering = math.atan2(right_dot, forward_dot) / math.pi
    control.steer = max(-1.0, min(1.0, steering))  # Clamp steering to [-1, 1]

    # Speed control
    speed_error = target_speed - speed
    if speed_error > 0:
        # If below target speed, apply throttle
        control.throttle = min(0.3, speed_error / target_speed)  # Reduced throttle limit to 0.3
        control.brake = 0.0
    else:
        # If above target speed, apply brake
        control.throttle = 0.0
        control.brake = min(0.3, -speed_error / target_speed)  # Reduced brake limit to 0.3

    return control

def drive_figure_eight(world, vehicle, waypoints, target_speed=10):
    """
    Drive the vehicle along the figure-8 track at a specified speed.
    
    Parameters:
    - world: CARLA world object.
    - vehicle: CARLA vehicle object.
    - waypoints: List of waypoints for the figure-8 track.
    - target_speed: Desired speed in km/h (default is 10 km/h).
    """
    try:
        for i, wp in enumerate(waypoints):
            print(f"Current target waypoint {i + 1}/{len(waypoints)}: {wp.transform.location}")
            
            # Draw the current target waypoint
            world.debug.draw_point(
                wp.transform.location,
                size=0.2,  # Size of the marker
                color=carla.Color(255, 0, 0),  # Red color
                life_time=5.0  # Persist for 5 seconds
            )
            
            # Drive toward the current waypoint
            while True:
                control = compute_control(vehicle, wp, target_speed)
                vehicle.apply_control(control)
                
                # Update spectator camera
                set_spectator_camera_following_car(world, vehicle)
                
                # Check if the vehicle is close to the current waypoint
                current_location = vehicle.get_transform().location
                distance_to_waypoint = current_location.distance(wp.transform.location)
                if distance_to_waypoint < 2.0:  # Proceed to the next waypoint if within 2 meters
                    break
                
                if world.get_settings().synchronous_mode:
                    world.tick()
                else:
                    time.sleep(0.1)  # Increased delay to 0.1 seconds
                    
    except KeyboardInterrupt:
        print("Simulation interrupted by user.")
        control = carla.VehicleControl(throttle=0, brake=1)
        vehicle.apply_control(control)

def draw_permanent_waypoint_lines(world, waypoints, color=carla.Color(0, 255, 0), thickness=0.01):
    """
    Draw permanent lines linking every waypoint on the road.
    
    Parameters:
    - world: CARLA world object.
    - waypoints: List of waypoints to link.
    - color: Color of the lines (default is neon green).
    - thickness: Thickness of the lines (default is 0.1 meters).
    """
    for i in range(len(waypoints) - 1):
        # Get the current and next waypoint
        wp1 = waypoints[i]
        wp2 = waypoints[i + 1]
        
        # Draw a line between the two waypoints
        world.debug.draw_line(
            wp1.transform.location,  # Start point
            wp2.transform.location,  # End point
            thickness=thickness,     # Line thickness
            color=color,             # Line color
            life_time=0             # Permanent line (life_time=0 means infinite)
        )        

In [47]:
waypoints = get_figure8_waypoints(world)
draw_permanent_waypoint_lines(world, waypoints) 

## Instantiate vehicle and place on start of figure of 8

In [55]:
def main():
    # Connect to the CARLA server
    client = carla.Client("localhost", 2000)
    client.set_timeout(10.0)
    world = client.get_world()

    # Enable synchronous mode
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.05
    world.apply_settings(settings)

    try:
        # Get waypoints for the figure-8 track
        waypoints = get_figure8_waypoints(world)
        print(f"Number of waypoints retrieved: {len(waypoints)}")

        # Draw permanent lines linking the waypoints
        draw_permanent_waypoint_lines(world, waypoints, color=carla.Color(0, 255, 255))  # Neon cyan

        # Spawn vehicle at the first waypoint
        blueprint_library = world.get_blueprint_library()
        vehicle_bp = blueprint_library.filter("vehicle.tesla.model3")[0]  # Choose a vehicle blueprint
        spawn_point = waypoints[0].transform
        spawn_point.location.z += 2.0  # Adjust spawn height to avoid collisions
        print(f"Vehicle spawn point: {spawn_point.location}")

        try:
            vehicle = world.spawn_actor(vehicle_bp, spawn_point)
            print(f"Vehicle spawned at {spawn_point.location}")
        except Exception as e:
            print(f"Failed to spawn vehicle: {e}")
            return  # Exit the script if spawning fails

        # Drive the vehicle around the figure-8 track at 10 km/h
        try:
            drive_figure_eight(world, vehicle, waypoints, target_speed=10)
        except Exception as e:
            print(f"Error during driving: {e}")
    except KeyboardInterrupt:
        print("Simulation interrupted by user.")
    finally:
        # Clean up
        if 'vehicle' in locals():
            vehicle.destroy()
            print("Vehicle destroyed.")

main()

Number of waypoints retrieved: 2356
Vehicle spawn point: Location(x=248.673141, y=-367.730164, z=2.000000)
Vehicle spawned at Location(x=248.673141, y=-367.730164, z=2.000000)
Current target waypoint 1/2356: Location(x=248.673141, y=-367.730164, z=0.000000)
Current target waypoint 2/2356: Location(x=249.673126, y=-367.735321, z=0.000000)
Current target waypoint 3/2356: Location(x=250.658020, y=-367.740265, z=0.000000)
Current target waypoint 4/2356: Location(x=251.574799, y=-367.740753, z=0.000000)
Current target waypoint 5/2356: Location(x=252.491562, y=-367.734985, z=0.000000)
Current target waypoint 6/2356: Location(x=253.408264, y=-367.723053, z=0.000000)
Current target waypoint 7/2356: Location(x=254.324875, y=-367.704834, z=0.000000)
Current target waypoint 8/2356: Location(x=255.241333, y=-367.680389, z=0.000000)
Current target waypoint 9/2356: Location(x=256.157593, y=-367.649780, z=0.000000)
Current target waypoint 10/2356: Location(x=257.073669, y=-367.612885, z=0.000000)
Cur